In [1]:
#%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite
#%pip install python-dotenv


In [2]:
#load the travily api key from .env file
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the Travily API key
travily_api_key = os.getenv("TRAVILY_API_KEY")

In [3]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1761117190, 'localtime': '2025-10-22 00:13'}, 'current': {'last_updated_epoch': 1761116400, 'last_updated': '2025-10-22 00:00', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 212, 'wind_dir': 'SSW', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 75, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 11.7, 'windchill_f': 53.1, 'heatindex_c': 12.9, 'heatindex_f': 55.3, 'dewpoint_c': 12.4, 

In [4]:
#%pip install -qU "langchain[google-genai]"

In [6]:
# Access the Travily API key
travily_api_key = os.getenv("GOOGLE_API_KEY")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

c:\Users\ASUS\anaconda3\envs\testone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

c:\Users\ASUS\anaconda3\envs\testone\Lib\site-packages\IPython\core\interactiveshell.py:3549: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .text as a property instead (e.g., message.text).
  exec(code_obj, self.user_global_ns, self.user_ns)


'Hello! How can I help you today?'

In [8]:
model_with_tools = model.bind_tools(tools)

In [10]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today?

Tool calls: []


In [12]:
query = "Search for the weather in SF"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in SF'}, 'id': 'fad2dfe3-e7fc-4ead-8d2a-1136ddab6e7b', 'type': 'tool_call'}]


In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_428\1543831842.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(model, tools)


In [14]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today?


In [16]:
input_message = {"role": "user", "content": "Search for the weather in Colombo"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in Colombo
================================== Ai Message ==================================
Tool Calls:
  tavily_search (2b5aa7d2-7fd0-4c49-acd5-0ee32fad690b)
 Call ID: 2b5aa7d2-7fd0-4c49-acd5-0ee32fad690b
  Args:
    query: weather in Colombo
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Colombo", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Colombo", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Colombo', 'region': 'Western', 'country': 'Sri Lanka', 'lat': 6.9319, 'lon': 79.8478, 'tz_id': 'Asia/Colombo', 'localtime_epoch': 1761119467, 'localtime': '2025-10-22 13:21'}, 'current': {'last_updated_epoch': 1761119100, 'last_updated': '2025-10-22 13:15', 'temp_c': 29.2, 'temp_f': 84.6, 'is_day': 1, 'condition': {'text': 'Overcast', 'ico

In [17]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in Colombo
================================== Ai Message ==================================
Tool Calls:
  tavily_search (89c1d485-d99b-47c3-a48a-8fe7a704be00)
 Call ID: 89c1d485-d99b-47c3-a48a-8fe7a704be00
  Args:
    query: weather in Colombo
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in Colombo", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Colombo", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Colombo', 'region': 'Western', 'country': 'Sri Lanka', 'lat': 6.9319, 'lon': 79.8478, 'tz_id': 'Asia/Colombo', 'localtime_epoch': 1761119467, 'localtime': '2025-10-22 13:21'}, 'current': {'last_updated_epoch': 1761119100, 'last_updated': '2025-10-22 13:15', 'temp_c': 29.2, 'temp_f': 84.6, 'is_day': 1, 'condition': {'text': 'Overcast', 'ico

In [19]:
config={}

for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

c:\Users\ASUS\anaconda3\envs\testone\Lib\site-packages\IPython\core\interactiveshell.py:3549: LangChainDeprecationWarning: Calling .text() as a method is deprecated. Use .text as a property instead (e.g., message.text).
  exec(code_obj, self.user_global_ns, self.user_ns)


The weather in Colombo|, Sri Lanka on October 22, 2025, at 13|:15 local time is 29.2°C (84.6°F) and feels like 35.4°C (95.8°F). It is overcast with a wind of 17.6| kph (11.0 mph) from the SW. The humidity is 79%, and there is 1.04mm of precipitation. The UV index is 8.1.|

In [20]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [24]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

C:\Users\ASUS\AppData\Local\Temp\ipykernel_428\4203846759.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(model, tools, checkpointer=memory)


In [25]:
for step in agent_executor.stream(
    {"messages": [("user", "Hi, I'm Bob!")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hello Bob! How can I help you today?


In [26]:
for step in agent_executor.stream(
    {"messages": [("user", "What is my name?")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

In this current conversation, you've told me your name is Bob.


In [27]:
config = {"configurable": {"thread_id": "xyz123"}}

for step in agent_executor.stream(
    {"messages": [("user", "What is my name?")]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is my name?
================================== Ai Message ==================================

I do not know your name. I am a large language model, able to communicate in response to a wide range of prompts and questions, but I have no memory of past interactions.
